In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('ITA_NIOT_nov16.xlsx', sheet_name='National IO-tables')

In [3]:
# Выбор года
df = df[(df['Year'] == 2000)]

# Удаление лишних строк
search_for = ['Imports', 'TOT']
mask = df.applymap(lambda x: any(s in str(x) for s in search_for))
rows_analysis = mask.any(axis=1)
rows_to_drop = rows_analysis[rows_analysis == True].index.tolist()
df_filtered = df.drop(rows_to_drop, axis=0)

#Сохранение названий отраслей
description = df_filtered['Description']

# Сохранение данных TOTAL_OUTPUT
total_output = df_filtered['GO']

# Замена нулевых значений в столбце TOTAL_OUTPUT
total_output = (total_output.replace(0, np.inf)).astype(float)

# Вектор конечных выпусков отраслей
w = df_filtered.loc[:, 'CONS_h':'GFCF']
w = w.sum(axis = 1)

# Удаление лишних столбцов
df_filtered = df_filtered.drop(['GO', 'Year', 'Code','Origin', 'Description','CONS_h', 'CONS_np', 'CONS_g', 'GFCF', 'INVEN', 'EXP'], axis = 1)

Создание матрицы $A = \|{ a_{ij}}\| = \frac{z_{ij}}{Y_{j}}$, где $z_{ij}$ $~-$ поставка продукции $i$-ой отрасли в $j$-ую отрасль в период времени $t$, $Y_{j}$ $~-$ объём производства $j$-ой отрасли в период времени $t$.

In [4]:
A = df_filtered.copy()
for i in df_filtered.columns:
    A[i] = df_filtered[i]/total_output
#print(A)
row_power = np.sum(A, axis = 0)
index_for_delete = []
for i in range(len(row_power)):
    if row_power[i] == 0:
        index_for_delete.append(i)
A = A.drop(A.columns[index_for_delete], axis = 1).drop(A.index[index_for_delete], axis = 0)
description = description.drop(description.index[index_for_delete])
#print(A)

Вектор конечного потребления $\overrightarrow{w}$

In [5]:
for i in index_for_delete:
    del w[i+1]

Проверка продуктивности (все последовательные главные миноры должны быть положительными):

In [6]:
for i in range(1, np.shape(A)[0] + 1):
    if (np.linalg.det(np.eye(i) - np.matrix(A, dtype = float)[:i,:i]) <= 0):
        print("Условие не выполняется!")
        break
    elif i == np.shape(A)[0]:
        print("Матрица продуктивная!")
    else:
        continue

Матрица продуктивная!


Вектор валового выпуска $\overrightarrow{x}$:

$$\overrightarrow{x} = A\overrightarrow{x} + \overrightarrow{w}$$
$$\overrightarrow{x} = (E - A)^{-1}\overrightarrow{w}$$

In [7]:
size = A.shape[0]
D = np.eye(size) - np.array(A, dtype = float)
w =  np.array(w)
w = w.reshape(54,1)
#print(w.shape, D.shape)
x = np.linalg.inv(D) @ w
#print(x)

In [8]:
#K = np.linalg.inv(D)
#(K[0] * w[:, 0]).sum()

## Вычисление вектора и числа Фробениуса – Перрона

Пусть $A \geqslant 0$ – матрица размером $n \times n$ . 
Выберем $x_0 \in \mathbb {R}^n$, $x_0 = (1,...,1)$ $~-$ начальное приближение собственного вектора $\overrightarrow{x}(A)$. Для каждой итерации $k > 0$ необходимо выполнить следующие действия: 
1. Вычисление очередного приближения для вектора Фробениуса – Перрона: $$x_{k+1} = Ax_{k}$$
2. Вычисление очередного приближения для числа Фробениуса – Перрона: 
$$\lambda^{k+1} = \frac{1}{n}\sum\limits_{i=1}^n\frac{x_i^{k+1}}{x_i^{k}}$$
3. Нормировка вектора $x_{k+1}$:
$$\widetilde{x}^{k+1} = \frac{1}{\max\limits_{i}{x_i^{k+1}}}{x}^{k+1}$$
4. Проверка условия достижения заданной точности вычислений: $|\lambda_{k +1} -\lambda_{k}| < \varepsilon$. Если заданная точность достигнута, то $\lambda_{k +1}$ и $\widetilde{x}^{k+1}$ взять в качестве искомых собственного числа и собственного вектора. В противном случае взять в качестве ${x}^{k}$ вектор $\widetilde{x}^{k+1}$ и перейти к очередной итерации. 

In [9]:
def find_FP(A, eps):
    A = np.matrix(A, dtype = float)
    n = A.shape[0]
    x_last = np.ones((n, 1))
    lambda_last = -1
    lambda_next = -1
    while ((abs(lambda_next - lambda_last) >= eps) or (lambda_next <= 0)):
        x_next = A @ x_last
        temp = x_next/x_last
        x_next = x_next/max(x_next)
        lambda_last = lambda_next
        lambda_next = sum(temp)/n;
        x_last = x_next
    res_lambda = lambda_next
    res_x = x_next
    return [res_lambda, res_x]

In [10]:
lambda_answ_A = find_FP(A, 0.01)[0]
x_answ_A = find_FP(A, 0.01)[1]
print('Число Фробениуса-Перрона для матрицы A:', lambda_answ_A)
print('Вектор Фробениуса-Перрона для матрицы A:\n', x_answ_A)

Число Фробениуса-Перрона для матрицы A: [[0.44579297]]
Вектор Фробениуса-Перрона для матрицы A:
 [[0.21169162]
 [0.5612926 ]
 [0.0749066 ]
 [0.87104567]
 [0.12572092]
 [0.11080189]
 [0.53324305]
 [0.71460607]
 [1.        ]
 [0.39276972]
 [0.42015032]
 [0.14364716]
 [0.43527251]
 [0.53704595]
 [0.69865731]
 [0.48858143]
 [0.16221521]
 [0.26010129]
 [0.21282725]
 [0.16015663]
 [0.14817538]
 [0.11998541]
 [0.35521808]
 [0.53125446]
 [0.26700288]
 [0.46654368]
 [0.17504926]
 [0.1987838 ]
 [0.47636943]
 [0.31375248]
 [0.54762197]
 [0.2398622 ]
 [0.29953383]
 [0.69459114]
 [0.6935228 ]
 [0.1654221 ]
 [0.35131079]
 [0.72000306]
 [0.50344261]
 [0.5353383 ]
 [0.49628768]
 [0.0839429 ]
 [0.57645259]
 [0.19927897]
 [0.70627233]
 [0.81484291]
 [0.3900579 ]
 [0.68736858]
 [0.65527431]
 [0.67748392]
 [0.00630962]
 [0.04996486]
 [0.00461036]
 [0.21575824]]


## Процедура исключения отраслей

Пусть имеется матрица прямых затрат $A = ||a_{ij}||$ размерности $n \times n$. Предположим, что для анализа нужны только r отраслей. Требуется найти матрицу прямых затрат $B = ||b_{ij}||$ размерностью $r \times r$ для выделенных отраслей.

In [11]:
save_industry = [5, 7, 10, 14, 18, 26, 37, 39, 48, 52]
r = len(save_industry)

In [12]:
#A

In [13]:
A_copy = A.to_numpy()

In [14]:
A11 = np.zeros((r, r))
m = -1
for i in save_industry:
    m += 1
    p = 0
    for k in save_industry:
        A11[m][p]  = A_copy[i-1][k-1]
        p += 1

In [15]:
index = [i-1 for i in save_industry]
n = A_copy.shape[0]
all_index = list([i for i in range(n)])
arr = list(set(all_index) - set(index)) # Индексы исключённых (n-r) отраслей

In [16]:
n = A_copy.shape[0]
A12 = np.zeros((r, n - r))
m = -1
for i in index:
    m += 1
    p = 0
    for k in arr:
        A12[m][p] = A_copy[i][k]
        p += 1

In [17]:
A21 = np.zeros((n - r, r))
m = -1
for i in arr:
    m += 1
    p = 0
    for k in index:
        A21[m][p] = A_copy[i][k]
        p += 1

In [18]:
A22 = np.zeros((n - r, n - r))
m = -1
for i in arr:
    m += 1
    p = 0
    for k in arr:
        A22[m][p] = A_copy[i][k]
        p += 1

$$B = A_{11}+A_{12}(E-A_{22})^{-1}A_{21}$$

In [19]:
B = A11 + A12 @ (np.eye(n - r) - A22) @ A21

In [20]:
#lambda_answ_B = find_FP(B, 0.01)[0]
#x_answ_B = find_FP(B, 0.01)[1]
#print('Число Фробениуса-Перрона для матрицы после исключения отраслей:\n', lambda_answ_B)
#print('Вектор Фробениуса-Перрона для матрицы после исключения отраслей:\n', x_answ_B)

In [21]:
size = B.shape[0]
D_B = np.eye(size) - B
w_B =  np.zeros((r, 1))
k = 0
for i in save_industry:
    w_B[k] = w[i]
    k += 1
x_B = np.linalg.inv(D_B) @ w_B
#print(x)

## Процедура агрегирования матрицы прямых затрат

Пусть имеется матрица прямых затрат $A = ||a_{ij}||$ размерности $n \times n$ и задан вектор валовых выпусков отраслей $\overrightarrow{x} = (x_1,...,x_n) \geqslant 0$. Предположим, что множество номеров отраслей $I =${$1,2,..,n$} разбито на $s$ непересекающихся подмножеств $I_1$, $I_2$,..., $I_s$. Каждое подмножество определяет номера отраслей, входящих в соответствующую укрупненную отрасль.

Требуется найти агрегированную матрицу $C = ||c_{ij}||$, размерностью $s \times s$ для укрупненных отраслей.

Коэффициенты матрицы $C = ||c_{ij}||$ вычисляются по формуле:
$$c_{kl} = \frac{\sum\limits_{i \in I_k}\sum\limits_{j \in I_l}a_{ij}x_{j}}{\sum\limits_{j \in I_l}x_j}$$

In [22]:
#description

In [23]:
#pip install googletrans==4.0.0rc1

In [24]:
from googletrans import Translator
translator = Translator()
#ar = translator.translate("Je t'aime", dest='russian').text
#print(ar)

In [25]:
russian_description = []
for i in description:
    str_ = translator.translate(i, dest = 'russian').text
    russian_description.append(str_)
#russian_description

In [26]:
list_index = [i for i in range(n)]
index_russian_description = [list_index[:4], list_index[4:26], list_index[26:41], list_index[41:]]

In [27]:
A_copy = A.to_numpy()
size_C = len(index_russian_description)
C = np.zeros((size_C, size_C))
for k in range(size_C):
    for l in range(size_C):
        list_ind_k = index_russian_description[k]
        list_ind_l = index_russian_description[l]
        for i in list_ind_k:
            for j in list_ind_l:
                C[k][l] += A_copy[i][j]*x[j]
        sum_xj = 0
        for j in list_ind_l:
            sum_xj += x[j]
        C[k][l] /= sum_xj
C

array([[0.02853058, 0.1040643 , 0.02792625, 0.00996803],
       [0.06571094, 0.30073234, 0.31005226, 0.12831219],
       [0.02634646, 0.11185507, 0.30195015, 0.16108375],
       [0.02354959, 0.08764485, 0.21258901, 0.15724998]])

In [28]:
for i in range(1, C.shape[0] + 1):
    if (np.linalg.det(np.eye(i) - np.matrix(C, dtype = float)[:i,:i]) <= 0):
        print("Условие не выполняется!")
        break
    elif i == np.shape(C)[0]:
        print("Матрица продуктивная!")
    else:
        continue

Матрица продуктивная!


In [29]:
lambda_answ_C = find_FP(C, 0.01)[0]
x_answ_C = find_FP(C, 0.01)[1]
print('Число Фробениуса-Перрона для агрегированной матрицы:\n', lambda_answ_C)
print('Вектор Фробениуса-Перрона для агрегированной матрицы:\n', x_answ_C)

Число Фробениуса-Перрона для агрегированной матрицы:
 [[0.60303245]]
Вектор Фробениуса-Перрона для агрегированной матрицы:
 [[0.22407284]
 [1.        ]
 [0.69407683]
 [0.54536582]]
